In [1]:
%set_env CUDA_DEVICE_ORDER=PCI_BUS_ID
%set_env CUDA_VISIBLE_DEVICES=1

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1


# import comet_ml in the top of your file
```
from comet_ml import Experiment

experiment = Experiment(api_key="jP4FiqPW37NTXkhiyzy9u1Jna", project_name="Alexnet_sourceOnly",
                        auto_param_logging=False, parse_args=False)
```

In [2]:
%matplotlib inline
from config import domainData
from config import num_classes as NUM_CLASSES
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision.models as models
import torchvision
import utils
from torchvision import datasets, transforms
import numpy as np
import itertools
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from alexnet import alexnet

net = models.alexnet(pretrained=True)
net.classifier = nn.Sequential(*list(net.classifier.children())[:-1])
# print(net)

In [3]:
net = alexnet
net.load_state_dict(torch.load('alexnet.pth'))
# print(net)

In [4]:
net = nn.Sequential(*list(net.children())[:-2])
print(net)

Sequential(
  (0): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4))
  (1): ReLU(inplace)
  (2): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=1)
  (3): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), dilation=(1, 1), ceil_mode=True)
  (4): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), groups=2)
  (5): ReLU(inplace)
  (6): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=1)
  (7): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), dilation=(1, 1), ceil_mode=True)
  (8): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (9): ReLU(inplace)
  (10): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=2)
  (11): ReLU(inplace)
  (12): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=2)
  (13): ReLU(inplace)
  (14): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), dilation=(1, 1), ceil_mode=True)
  (15): Lambda(
  )
  (16): Sequential(
    (0): Lambda(
    )
    (1): Linear(in_features=9216, out_features=4096, bias

In [5]:
src = domainData['amazon']
tar = domainData['webcam']

In [6]:
src_transforms = transforms.Compose([
    transforms.Resize(256),
#     transforms.CenterCrop(227),
#     transforms.RandomVerticalFlip(),
    transforms.RandomResizedCrop(227, scale=(0.50,1.0), ratio=(1.,1.)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[1.,1.,1.]),
    transforms.Lambda(lambda x: torch.stack([x[2],x[1],x[0]])), # RGB -> BGR
    transforms.Lambda(lambda x: x * 255.)
])
tar_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(227),
#     transforms.RandomResizedCrop(224, scale=(0.25,1.0)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[1.,1.,1.]),
    transforms.Lambda(lambda x: torch.stack([x[2],x[1],x[0]])),
    transforms.Lambda(lambda x: x * 255.)
])

In [7]:
src_dataset = datasets.ImageFolder(src, transform=src_transforms)
tar_dataset = datasets.ImageFolder(tar, transform=tar_transforms)
srcSampler = torch.utils.data.sampler.RandomSampler(src_dataset)
tarSampler = torch.utils.data.sampler.RandomSampler(tar_dataset)
srcDataLen = len(src_dataset)
tarDataLen = len(tar_dataset)
use_gpu = True and torch.cuda.is_available()

In [8]:
opt = {
    'src': 'Amazon',
    'tar': 'Webcam',
    'manual_seed':1,
    'batchSize':32,
    'use_gpu': use_gpu,
    'num_classes': 31,
    'epochs': 75,
    'momentum': 0.9,
    'lr': 2e-4,
    'lr_sch': 0,
    'lr_sch_gamma': 0.1,
    'p_lr_decay': 10000,
    'n0': 1.,
    'alpha': 10,
    'beta': 0.75,
    'betas': (0.5,0.99),
    'net_wtDcy': 0.001,
    'btl_wtDcy': 0.001,
    'srcDataLen': srcDataLen,
    'tarDataLen': tarDataLen
}

experiment.log_multiple_params(opt)

In [9]:
torch.manual_seed(opt['manual_seed'])
if opt['use_gpu']: torch.cuda.manual_seed(opt['manual_seed'])

In [10]:
print("use_gpu: ", opt['use_gpu'])

use_gpu:  True


In [11]:
src_dataloader = torch.utils.data.DataLoader(src_dataset, batch_size=opt['batchSize'], 
                                             shuffle=(srcSampler is None), sampler=srcSampler,
                                            num_workers=2, pin_memory=True, drop_last=False)
tar_dataloader = torch.utils.data.DataLoader(tar_dataset, batch_size=opt['batchSize'],
                                            shuffle=(tarSampler is None), sampler=tarSampler,
                                            num_workers=2, pin_memory=True, drop_last=False)

In [12]:
def init_weights(m):
#     print(m)
    if isinstance(m, nn.Linear):
        init.xavier_normal(m.weight)
        init.constant(m.bias, 0.1)

In [13]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.classifier = nn.Sequential(nn.Linear(4096,256),
                                       nn.ReLU(inplace=True), nn.Linear(256, opt['num_classes']))
        self.classifier.apply(init_weights)
    
    def forward(self, x):
        x = self.classifier(x)
        return x

In [14]:
net2 = Model()

In [15]:
if opt['use_gpu']:
    net = net.cuda()
    net2 = net2.cuda()
    torch.backends.cudnn.enabled=True
    torch.backends.cudnn.benchmark=True

To computer norm of various parameters

```python
for name, param in net2.named_parameters():
    nrm = torch.norm(param, 2)
    zero = param.eq(0.).float().sum()
    nele = torch.numel(param)
    print(name, nrm.data[0], nele, zero.data[0])
```

In [16]:
criterion = nn.CrossEntropyLoss()
softmax = nn.Softmax(dim=1)

In [17]:
netfeatwt, netfeatbias = list(), list()
for name, param in net.named_parameters():
    if name in ['10.weight', '12.weight', '16.1.weight', '19.1.weight']: netfeatwt.append(param)
    elif name in ['10.bias', '12.bias', '16.1.bias', '19.1.bias']: netfeatbias.append(param)
    else: param.requires_grad=False

In [18]:
for name, param in net.named_parameters():
    print(name, param.requires_grad)

0.weight False
0.bias False
4.weight False
4.bias False
8.weight False
8.bias False
10.weight True
10.bias True
12.weight True
12.bias True
16.1.weight True
16.1.bias True
19.1.weight True
19.1.bias True


In [19]:
net2_weight, net2_bias = list(), list()
for name, param in net2.named_parameters():
    if 'weight' in name: net2_weight.append(param)
    elif 'bias' in name: net2_bias.append(param)

In [20]:
sgd_params = [
    {'params': netfeatwt, 'lr': opt['lr'], 'momentum': opt['momentum'], 'weight_decay': opt['net_wtDcy'], 'lr_mul': 1., 'name': 'netfeatwt'},
    {'params': netfeatbias, 'lr': opt['lr'], 'momentum': opt['momentum'], 'weight_decay': opt['net_wtDcy'], 'lr_mul': 2., 'name': 'netfeatbias'},
    {'params': net2_weight, 'lr': opt['lr'], 'momentum': opt['momentum'], 'weight_decay': opt['btl_wtDcy'], 'lr_mul': 10., 'name': 'net2wt'},
    {'params': net2_bias, 'lr': opt['lr'], 'momentum': opt['momentum'], 'weight_decay': opt['btl_wtDcy'], 'lr_mul': 20., 'name': 'net2bias'}
]

# optimizer = optim.Adam(net2.parameters(), lr=opt['lr'], betas=opt['betas'], weight_decay=opt['weight_decay'])
# optimizer = optim.Adam(sgd_params, betas=opt['betas'])

optimizer_2 = optim.SGD(sgd_params)
# optimizer_2 = optim.SGD(net2.parameters(), lr=opt['lr'], momentum=opt['momentum'], weight_decay=opt['btl_wtDcy'])

lr_sch = None
if opt['lr_sch']=='step': lr_sch = optim.lr_scheduler.StepLR(optimizer, 20, opt['lr_sch_gamma'])
if opt['lr_sch']=='exponential': lr_sch = optim.lr_scheduler.ExponentialLR(optimizer_2, opt['lr_sch_gamma'])

In [21]:
def get_validation_acc(cm=False):
    tarData = iter(tar_dataloader)
    totalCorrects = 0.
    if cm: y_preds, y_true = list(), list()
    for tarimgs, tarlbls in tarData:
        tarimgs = tarimgs.cuda() if opt['use_gpu'] else tarimgs
        tarlbls = tarlbls.cuda() if opt['use_gpu'] else tarlbls      
        tarimgs = Variable(tarimgs, volatile=True)

        feat_ = net(tarimgs)
        logits = net2(feat_)

        _, preds = torch.max(softmax(logits).data, 1)
        totalCorrects += torch.eq(preds, tarlbls).float().sum()
        if cm: y_preds.append(preds.cpu().numpy()), y_true.append(tarlbls.cpu().numpy())
    valAcc = totalCorrects / opt['tarDataLen']
    if cm: return valAcc, y_preds, y_true
    return valAcc

In [22]:
p = np.linspace(float(1./opt['p_lr_decay']),1,opt['p_lr_decay'])

for epoch in range(opt['epochs']):
    srcData = iter(src_dataloader)
    totalCorrects = 0.
    totalClsLoss = 0.
#     experiment.log_current_epoch(epoch)
    
    n_p = opt['n0'] / pow((1. + opt['alpha'] * p[epoch]), (opt['beta']))
    print("n_p: ", n_p)
    for param_group in optimizer_2.param_groups:
        param_group['lr'] = opt['lr'] * param_group['lr_mul'] * n_p
        
    for srcimgs, srclbls in srcData:
        srcimgs = srcimgs.cuda() if opt['use_gpu'] else srcimgs
        srclbls = srclbls.cuda() if opt['use_gpu'] else srclbls
        srcimgs, srclbls = Variable(srcimgs), Variable(srclbls)
        
        feat_ = net(srcimgs)
        logits = net2(feat_)
        
        clsloss = criterion(logits, srclbls)
        totalClsLoss += clsloss.data[0] * opt['batchSize']
        
        _, preds = torch.max(softmax(logits).data, 1)
        totalCorrects += torch.eq(preds, srclbls.data).float().sum()
               
        optimizer_2.zero_grad()
        clsloss.backward()
        optimizer_2.step()
        
        if lr_sch: lr_sch.step()
        
    srcAcc = totalCorrects / opt['srcDataLen']
    srcLoss = totalClsLoss / opt['srcDataLen']
    valAcc = get_validation_acc()
#     experiment.log_metric("src_clsLoss", srcLoss, step=epoch)
#     experiment.log_metric("src_clsAcc", srcAcc, step=epoch)
#     experiment.log_metric('val_accuracy', valAcc, step=epoch)
#     experiment.log_epoch_end(epoch)
    print("Epoch: {}, train loss: {:.4f}, train acc: {:.4f}, validation acc: {:.4f}".
          format(epoch+1, srcLoss, srcAcc, valAcc))

n_p:  0.999250655649
Epoch: 1, train loss: 2.1570, train acc: 0.4860, validation acc: 0.4931
n_p:  0.998502620197
Epoch: 2, train loss: 1.0631, train acc: 0.7149, validation acc: 0.4956
n_p:  0.997755890053
Epoch: 3, train loss: 0.8594, train acc: 0.7526, validation acc: 0.5019
n_p:  0.997010461644
Epoch: 4, train loss: 0.7147, train acc: 0.7938, validation acc: 0.4994
n_p:  0.996266331405
Epoch: 5, train loss: 0.6553, train acc: 0.8129, validation acc: 0.5094
n_p:  0.995523495789
Epoch: 6, train loss: 0.5549, train acc: 0.8364, validation acc: 0.5283
n_p:  0.994781951259
Epoch: 7, train loss: 0.5478, train acc: 0.8314, validation acc: 0.5119
n_p:  0.994041694293
Epoch: 8, train loss: 0.4451, train acc: 0.8701, validation acc: 0.5447
n_p:  0.99330272138
Epoch: 9, train loss: 0.3777, train acc: 0.8832, validation acc: 0.5358
n_p:  0.992565029024
Epoch: 10, train loss: 0.3469, train acc: 0.8931, validation acc: 0.5132
n_p:  0.991828613742
Epoch: 11, train loss: 0.2874, train acc: 0.9070,

n_p:  0.938056701012
Epoch: 89, train loss: 0.0342, train acc: 0.9901, validation acc: 0.5447
n_p:  0.937411175105
Epoch: 90, train loss: 0.0341, train acc: 0.9886, validation acc: 0.5673
n_p:  0.936766684762
Epoch: 91, train loss: 0.0259, train acc: 0.9915, validation acc: 0.5421
n_p:  0.936123227375
Epoch: 92, train loss: 0.0314, train acc: 0.9908, validation acc: 0.5447
n_p:  0.935480800343
Epoch: 93, train loss: 0.0203, train acc: 0.9943, validation acc: 0.5497
n_p:  0.934839401078
Epoch: 94, train loss: 0.0237, train acc: 0.9915, validation acc: 0.5396
n_p:  0.934199026996
Epoch: 95, train loss: 0.0335, train acc: 0.9872, validation acc: 0.5409
n_p:  0.933559675527
Epoch: 96, train loss: 0.0222, train acc: 0.9918, validation acc: 0.5346
n_p:  0.932921344106
Epoch: 97, train loss: 0.0330, train acc: 0.9890, validation acc: 0.5610
n_p:  0.932284030177
Epoch: 98, train loss: 0.0253, train acc: 0.9936, validation acc: 0.5585
n_p:  0.931647731195
Epoch: 99, train loss: 0.0258, train ac

n_p:  0.885512117842
Epoch: 176, train loss: 0.0115, train acc: 0.9972, validation acc: 0.5371
n_p:  0.884947797837
Epoch: 177, train loss: 0.0085, train acc: 0.9982, validation acc: 0.5472
n_p:  0.884384316257
Epoch: 178, train loss: 0.0161, train acc: 0.9961, validation acc: 0.5308
n_p:  0.883821671147
Epoch: 179, train loss: 0.0200, train acc: 0.9943, validation acc: 0.5396
n_p:  0.883259860556
Epoch: 180, train loss: 0.0141, train acc: 0.9936, validation acc: 0.5472
n_p:  0.882698882542
Epoch: 181, train loss: 0.0208, train acc: 0.9933, validation acc: 0.5371
n_p:  0.882138735167
Epoch: 182, train loss: 0.0127, train acc: 0.9965, validation acc: 0.5308
n_p:  0.881579416499
Epoch: 183, train loss: 0.0129, train acc: 0.9957, validation acc: 0.5195
n_p:  0.881020924615
Epoch: 184, train loss: 0.0116, train acc: 0.9965, validation acc: 0.5472
n_p:  0.880463257595
Epoch: 185, train loss: 0.0145, train acc: 0.9957, validation acc: 0.5296
n_p:  0.879906413526
Epoch: 186, train loss: 0.016

n_p:  0.839358488966
Epoch: 263, train loss: 0.0131, train acc: 0.9961, validation acc: 0.5447
n_p:  0.83886040262
Epoch: 264, train loss: 0.0088, train acc: 0.9972, validation acc: 0.5610
n_p:  0.838363005394
Epoch: 265, train loss: 0.0049, train acc: 0.9982, validation acc: 0.5547
n_p:  0.837866295791
Epoch: 266, train loss: 0.0082, train acc: 0.9972, validation acc: 0.5635
n_p:  0.837370272319
Epoch: 267, train loss: 0.0125, train acc: 0.9957, validation acc: 0.5472
n_p:  0.83687493349
Epoch: 268, train loss: 0.0067, train acc: 0.9979, validation acc: 0.5585
n_p:  0.836380277819
Epoch: 269, train loss: 0.0066, train acc: 0.9986, validation acc: 0.5509
n_p:  0.835886303827
Epoch: 270, train loss: 0.0113, train acc: 0.9972, validation acc: 0.5686
n_p:  0.83539301004
Epoch: 271, train loss: 0.0076, train acc: 0.9979, validation acc: 0.5623
n_p:  0.834900394986
Epoch: 272, train loss: 0.0076, train acc: 0.9975, validation acc: 0.5522
n_p:  0.834408457199
Epoch: 273, train loss: 0.0107, 

n_p:  0.79845358214
Epoch: 350, train loss: 0.0052, train acc: 0.9989, validation acc: 0.5497
n_p:  0.798010284131
Epoch: 351, train loss: 0.0059, train acc: 0.9982, validation acc: 0.5547
n_p:  0.79756755997
Epoch: 352, train loss: 0.0039, train acc: 0.9986, validation acc: 0.5484
n_p:  0.797125408491
Epoch: 353, train loss: 0.0051, train acc: 0.9982, validation acc: 0.5434
n_p:  0.796683828531
Epoch: 354, train loss: 0.0018, train acc: 1.0000, validation acc: 0.5723
n_p:  0.79624281893
Epoch: 355, train loss: 0.0088, train acc: 0.9965, validation acc: 0.5447
n_p:  0.795802378531
Epoch: 356, train loss: 0.0072, train acc: 0.9986, validation acc: 0.5396
n_p:  0.79536250618
Epoch: 357, train loss: 0.0041, train acc: 0.9989, validation acc: 0.5296
n_p:  0.794923200728
Epoch: 358, train loss: 0.0060, train acc: 0.9972, validation acc: 0.5371
n_p:  0.794484461026
Epoch: 359, train loss: 0.0045, train acc: 0.9989, validation acc: 0.5421
n_p:  0.794046285931
Epoch: 360, train loss: 0.0099, t

n_p:  0.761916579262
Epoch: 437, train loss: 0.0162, train acc: 0.9961, validation acc: 0.5459
n_p:  0.761519161236
Epoch: 438, train loss: 0.0042, train acc: 0.9989, validation acc: 0.5396
n_p:  0.76112222656
Epoch: 439, train loss: 0.0058, train acc: 0.9982, validation acc: 0.5660
n_p:  0.760725774313
Epoch: 440, train loss: 0.0053, train acc: 0.9986, validation acc: 0.5522
n_p:  0.760329803572
Epoch: 441, train loss: 0.0039, train acc: 0.9982, validation acc: 0.5459
n_p:  0.75993431342
Epoch: 442, train loss: 0.0059, train acc: 0.9986, validation acc: 0.5572
n_p:  0.759539302941
Epoch: 443, train loss: 0.0032, train acc: 0.9989, validation acc: 0.5270
n_p:  0.759144771221
Epoch: 444, train loss: 0.0046, train acc: 0.9975, validation acc: 0.5635
n_p:  0.758750717349
Epoch: 445, train loss: 0.0045, train acc: 0.9982, validation acc: 0.5308
n_p:  0.758357140416
Epoch: 446, train loss: 0.0064, train acc: 0.9986, validation acc: 0.5434
n_p:  0.757964039516
Epoch: 447, train loss: 0.0085,

```
valAcc, y_preds, y_true = get_validation_acc(cm=True)

y_preds = [d for sublist in y_preds for d in sublist]
y_true = [d for sublist in y_true for d in sublist]

cm = confusion_matrix(y_true, y_preds)

np.set_printoptions(precision=2)
plt.figure(figsize=(20,20))
utils.plot_confusion_matrix(cm, classes=src_dataset.classes, normalize=False)
plt.show()
```